<h1>Logistic regression weight update process</h1>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
# Generate synthetic data
np.random.seed(0)
centers = [[-5, 0], [0, 1.5]]
X, y = make_blobs(n_samples=1000, centers=centers, random_state=40)
transformation = [[0.4, 0.2], [-0.4, 1.2]]
X = np.dot(X, transformation)
# Add a bias term to the feature matrix
X = np.c_[np.ones((X.shape[0], 1)), X]
# Initialize coefficients
W = np.zeros(X.shape[1])
# Define the logistic sigmoid function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))
# Define the logistic loss ( binary cross - entropy ) function
def log_loss(y_true, y_pred):
    epsilon = 1e-15
    y_pred = np.clip(y_pred, epsilon, 1-epsilon) # Clip to avoid log (0)
    return -(y_true * np.log(y_pred) + (1 - y_true) * np.log (1 - y_pred ))
# Gradient descent and Newton method parameters
learning_rate = 0.1
iterations = 10
loss_history = []

In [2]:
# Performing Gradient Descent
for _ in range(iterations):
    # Computing the predicted probabilities using the sigmoid function
    y_pred = sigmoid(np.dot(X, W))
    
    # Computing the gradient of the loss function
    gradient = (1 / len(y)) * np.dot(X.T, (y_pred - y))
    
    # Updating the weights using the gradient
    W -= learning_rate * gradient
    
    # Calculating the loss and append it to the loss history
    loss = np.mean(log_loss(y, y_pred))
    loss_history.append(loss)


<h1>Perform grid search for hyperparameter tuning</h1>

In [28]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.utils import check_random_state
# data loading
train_samples = 500
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False)
random_state = check_random_state(0)
permutation = random_state.permutation(X.shape[0])
X = X[permutation]
y = y[permutation]
X = X.reshape((X.shape[0], -1))
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_samples, test_size=100)


c:\Python311\Lib\site-packages\sklearn\datasets\_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(
c:\Python311\Lib\site-packages\sklearn\datasets\_openml.py:67: RuntimeWarning: Invalid cache, redownloading file
  warn("Invalid cache, redownloading file", RuntimeWarning)


KeyboardInterrupt: 